# VerbaLex ASR Model [AR]

This notebook is a training environment to train a Whisper model in recognizing Arabic accent for English speech.

# Prepare Environment


In [1]:
!pip install --upgrade pip
!pip install --upgrade accelerate soundfile librosa evaluate jiwer tensorboard gradio transformers
!pip install datasets==2.16.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=e3dd4503a738f9e9441239d

# Linking to Huggingface Hub
Linking this notebook to huggingface hub to enjoy these benefits:
* Integrated version control: you can be sure that no model checkpoint is lost during training.
* Tensorboard logs: track important metrics over the course of training.
* Model cards: document what a model does and its intended use cases.
* Community: an easy way to share and collaborate with the community!

This step is also **required** as we need authentication to use a dataset from Huggingface later to fine tune our base model.

To get the access token for login, head over to your Huggingface profile and press on`Settings`, then `Access Tokens`. Copy your tokens and add it as a secret variable in the notebook's secrets. Steps on how to add your token to the secrets variable are as below.

1. Go to `Add-ons` on the toolbar at the top.
2. Press on `Secrets`.
3. Press on `Add a new secret`.
4. Name your secret variable `HF_TOKEN`, and paste in the access token copied from Huggingface.

If you would like to name your secret variable other names than `HF_TOKEN`, feel free to do so, but make sure to change the name in the `get_secret()` function in the code below.

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HF_TOKEN"))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load dataset



In [3]:
from datasets import load_dataset, DatasetDict

dataset = DatasetDict()

dataset["train"] = load_dataset("RitchieP/VerbaLex_voice", "ar", split="train")
dataset["test"] = load_dataset("RitchieP/VerbaLex_voice", "ar", split="test")
dataset

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for RitchieP/VerbaLex_voice contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/RitchieP/VerbaLex_voice
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3054it [00:00, 161295.01it/s]


Generating test split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1311it [00:00, 181668.18it/s]
/opt/conda/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for RitchieP/VerbaLex_voice contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/RitchieP/VerbaLex_voice
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['path', 'sentence', 'audio'],
        num_rows: 3054
    })
    test: Dataset({
        features: ['path', 'sentence', 'audio'],
        num_rows: 1311
    })
})

Let's have a peek of one row of the dataset

In [4]:
dataset["train"][0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/beb08b06b235fbe2cebdc53b22490cae664575e559ad035eb58c34f43736989f/ar_train/ABA_arctic_a0004.wav',
 'sentence': "Lord but I'm glad to see you again Phil",
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/beb08b06b235fbe2cebdc53b22490cae664575e559ad035eb58c34f43736989f/ar_train/ABA_arctic_a0004.wav',
  'array': array([-0.00311279, -0.00234985, -0.00305176, ..., -0.00198364,
         -0.00244141, -0.0015564 ]),
  'sampling_rate': 44100}}

## Data Cleaning
From the dataset, we will only select the columns `audio` and `sentence`

In [5]:
dataset = dataset.select_columns(['audio', 'sentence'])
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3054
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1311
    })
})

## Prepare Feature Extractor, Tokenizer and Data

In [6]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

processor = WhisperProcessor.from_pretrained("openai/whisper-small", langauge="English", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Testing
Test to see if the encoder and decoder works or not

In [7]:
input_str = dataset["train"][0]["sentence"]

labels = tokenizer(input_str).input_ids

decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

2024-04-04 08:19:33.708579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 08:19:33.708681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 08:19:33.849914: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Input:                 Lord but I'm glad to see you again Phil
Decoded w/out special: Lord but I'm glad to see you again Phil
Are equal:             True


## Data Preprocessing
Since the Whisper model was trained on data with a sampling rate of 16,000, we'll need to resample our data's sampling rate because the data provided by L2-Arctic Corpus has a sampling rate of 44,100kHz.

In [8]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

dataset["train"][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/beb08b06b235fbe2cebdc53b22490cae664575e559ad035eb58c34f43736989f/ar_train/ABA_arctic_a0004.wav',
  'array': array([-0.00177099, -0.00355739, -0.00425776, ..., -0.0012211 ,
         -0.00265928,  0.        ]),
  'sampling_rate': 16000},
 'sentence': "Lord but I'm glad to see you again Phil"}

After the sampling rate of our data has been resampled to 16,000kHz, we can then proceed to prepare our data for the model.

In [9]:
def prepare_dataset(batch):
    audio = batch["audio"]
    
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"])
dataset["train"][0]

Map:   0%|          | 0/3054 [00:00<?, ? examples/s]

Map:   0%|          | 0/1311 [00:00<?, ? examples/s]

{'input_features': [[0.19155752658843994,
   0.6788527965545654,
   0.4647698402404785,
   0.6748826503753662,
   0.7344000339508057,
   0.6709499359130859,
   0.40658658742904663,
   0.6011510491371155,
   0.7022650241851807,
   0.08371996879577637,
   -0.0037671327590942383,
   0.68654465675354,
   0.5538336038589478,
   0.2586004137992859,
   0.5672218799591064,
   0.7001083493232727,
   0.6714267134666443,
   0.6089007258415222,
   0.6013200283050537,
   0.6773097515106201,
   0.6577988266944885,
   0.6540607810020447,
   0.4474431276321411,
   0.5035933256149292,
   0.3737753629684448,
   0.6012566089630127,
   0.735680103302002,
   0.7516541481018066,
   0.6575982570648193,
   0.4618937373161316,
   0.622797966003418,
   0.6023639440536499,
   0.44715601205825806,
   0.4098854660987854,
   0.5882521271705627,
   0.5553323030471802,
   0.07296484708786011,
   0.5169761180877686,
   0.579883873462677,
   0.5441805124282837,
   0.5524778366088867,
   0.5891391634941101,
   0.5829884

In [10]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [11]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

## Load a pre-trained checkpoint

In [12]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

processor = WhisperProcessor.from_pretrained("openai/whisper-small")

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")

model.generation_config.language = "<|en|>"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

## Define training arguments

In [13]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-eng-ar",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Training


In [15]:
trainer.train() 

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.001800,0.081002,3.747958
2000,0.000200,0.085130,3.490071
3000,0.000100,0.086888,3.438494
4000,0.000100,0.087900,3.447090
5000,0.000100,0.088284,3.481475


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint

TrainOutput(global_step=5000, training_loss=0.07829960962221957, metrics={'train_runtime': 40082.6, 'train_samples_per_second': 1.996, 'train_steps_per_second': 0.125, 'total_flos': 2.307182558478336e+19, 'train_loss': 0.07829960962221957, 'epoch': 26.18})